# V21: Sector consistency (depth stability across sectors)

Compares per-sector transit depth measurements to test whether the signal is consistent across observing campaigns.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

import bittr_tess_vetter.api as btv

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

# Build per-sector depth measurements (simple in/out-of-transit estimator).
sector_measurements = []
for sector, lc_s in sorted(ds.lc_by_sector.items()):
    in_tr = btv.get_in_transit_mask(lc_s.time, sh.PERIOD_DAYS, sh.T0_BTJD, sh.DURATION_HOURS)
    out_tr = btv.get_out_of_transit_mask(lc_s.time, sh.PERIOD_DAYS, sh.T0_BTJD, sh.DURATION_HOURS, buffer_factor=3.0)
    d, de = btv.measure_transit_depth(lc_s.flux, in_tr, out_tr)
    sector_measurements.append({'sector': int(sector), 'depth_ppm': float(d*1e6), 'depth_err_ppm': float(de*1e6)})

session = sh.make_session(
    stitched=lc,
    candidate=candidate,
    network=False,
    preset='extended',
    context={'sector_measurements': sector_measurements},
)


r = session.run('V21')

print(json.dumps({'status': r.status, 'flags': list(r.flags), 'metrics': r.metrics}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "chi2_p_value": 7.276181265682546e-07,
    "consistency_class": "INCONSISTENT",
    "n_outlier_sectors": 2,
    "n_sectors_used": 4
  },
  "status": "ok"
}
```

</details>


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_sector_consistency

step_id = '23_v21_sector_consistency'
fname = 'V21_sector_consistency.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    plot_sector_consistency(r, ax=ax)
    ax.set_title('V21: Sector consistency (depth stability across sectors)')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/23_v21_sector_consistency/V21_sector_consistency.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/23_v21_sector_consistency/V21_sector_consistency.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/23_v21_sector_consistency/V21_sector_consistency.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** χ² p-value ≪ 0.01 and classification `INCONSISTENT`, with outlier sectors flagged in the underlying result.
- **Why this is useful:** strong cross-sector depth inconsistency is a major warning for systematics, aperture/contamination sensitivity, or sector-specific extraction issues.
- **Interpretation:** treat as high-priority caution; do not average all sectors blindly.
- **Next step:** choose a gated sector set (e.g., excluding outliers) and re-run V11–V20 and V16–V19 on that gated dataset to see whether warnings persist.

</details>
